# CareMap: AI-Powered Caregiver Fridge Sheet

**MedGemma Impact Challenge Submission**

---

## The Problem

Family caregivers receive overwhelming amounts of medical information at discharge:
- Dense clinical notes with medical jargon
- Multiple medication instructions
- Lab results with unexplained abbreviations
- Follow-up tasks scattered across documents

**Result:** Confusion, missed appointments, medication errors, and caregiver burnout.

## The Solution: CareMap

CareMap transforms structured health data into a **single-page "fridge sheet"** that caregivers can:
- Stick on the refrigerator for quick reference
- Understand at a 6th-grade reading level
- Use to track daily priorities and medications
- Share with family members involved in care

```
+------------------------------------------+
|     CAREMAP FRIDGE SHEET - Dadu          |
+------------------------------------------+
| TODAY'S PRIORITIES                       |
| [ ] Get flu shot at pharmacy             |
| [ ] Weigh yourself - write it down       |
+------------------------------------------+
| MEDICATIONS (8 total)                    |
| Warfarin - evening - blood thinner       |
| Metformin - with meals - diabetes        |
| ...                                      |
+------------------------------------------+
| RECENT LABS                              |
| Clotting level: Needs follow-up          |
| Kidney function: Stable                  |
+------------------------------------------+
| CONTACTS                                 |
| Clinic: 555-0100  Pharmacy: 555-0200     |
+------------------------------------------+
```

## Model Attribution

This project uses **MedGemma 4B-IT**, a medical foundation model from Google Health AI.

```bibtex
@article{sellergren2025medgemma,
  title={MedGemma Technical Report},
  author={Sellergren, Andrew and others},
  journal={arXiv preprint arXiv:2507.05201},
  year={2025}
}
```

**Safety by Design:**
- No diagnosis or treatment recommendations
- No dosage instructions or changes
- Plain language only (no medical jargon)
- Questions for the doctor, not answers

---

**Interactive Demo:** [CareMap on HuggingFace Spaces](https://huggingface.co/spaces/YOUR_USERNAME/caremap) *(link to be added)*

## Environment Setup

In [ ]:
# Install dependencies (run once)
# Uncomment the line below if running on Kaggle or fresh environment

# !pip install -q transformers>=4.39.0 accelerate>=0.27.0 huggingface-hub>=0.34.0 pydantic>=2.6.0 sentencepiece tqdm

In [ ]:
import json
import os
import sys
from pathlib import Path
from typing import Dict, Any, List
from IPython.display import display, Markdown, HTML

# Add src to path for imports
project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(project_root / 'src'))

print(f"Project root: {project_root}")
print(f"Python version: {sys.version}")

## Load Golden Patient Data

We use a realistic synthetic patient case: **"Dadu"** - an 80-year-old man with:
- Type 2 Diabetes with kidney complications
- Congestive Heart Failure
- Atrial Fibrillation (on blood thinners)
- 8 active medications with interactions
- 6 recent lab results
- 5 care gaps requiring action

In [ ]:
# Load the golden patient data
golden_file = project_root / 'examples' / 'golden_patient_complex.json'

with open(golden_file) as f:
    patient_data = json.load(f)

print(f"Patient: {patient_data['patient']['nickname']}")
print(f"Age: {patient_data['patient']['age_range']}")
print(f"Conditions: {', '.join(patient_data['patient']['conditions_display'])}")
print(f"\nMedications: {len(patient_data['medications'])}")
print(f"Lab Results: {len(patient_data['results'])}")
print(f"Care Gaps: {len(patient_data['care_gaps'])}")

In [ ]:
# Display the medications with their complexities
print("MEDICATIONS WITH INTERACTIONS:")
print("=" * 60)
for med in patient_data['medications']:
    print(f"\n{med['medication_name']} ({med['timing']})")
    if med.get('interaction_notes'):
        print(f"   Interactions: {med['interaction_notes'][:80]}...")

## Initialize MedGemma

We use the CareMap wrapper around MedGemma which:
- Handles GPU/CPU detection automatically
- Uses optimized generation parameters
- Integrates with our safety-focused prompts

In [ ]:
from caremap.llm_client import MedGemmaClient

print("Initializing MedGemma...")
print("(This may take 1-2 minutes on first run)")

client = MedGemmaClient()

print(f"\nModel: {client.model_id}")
print(f"Device: {client.device}")
print("Ready!")

---

## Domain 1: Lab Interpretation

CareMap explains lab results in plain language without:
- Numeric values or reference ranges
- Medical abbreviations (INR, BNP, eGFR)
- Diagnostic conclusions

Instead, it provides:
- What the test checks (in plain words)
- What the result means (qualitative)
- A question to ask the doctor

In [ ]:
from caremap.lab_interpretation import interpret_lab, LAB_OUT_KEYS
from tqdm import tqdm

print("Interpreting lab results...")
print("=" * 60)

lab_results = []

for lab in tqdm(patient_data['results'][:4], desc="Labs"):  # First 4 labs
    result = interpret_lab(
        client=client,
        test_name=lab['test_name'],
        meaning_category=lab['meaning_category'],
        source_note=lab.get('source_note', ''),
    )
    lab_results.append({
        'test_name': lab['test_name'],
        'flag': lab['flag'],
        'interpretation': result
    })

print("\nDone!")

In [ ]:
# Display lab interpretations
for lab in lab_results:
    print(f"\n{'='*60}")
    print(f"TEST: {lab['test_name']} ({lab['flag']})")
    print(f"{'='*60}")
    interp = lab['interpretation']
    print(f"What was checked: {interp.get('what_was_checked', 'N/A')}")
    print(f"What it means: {interp.get('what_it_means', 'N/A')}")
    print(f"Ask the doctor: {interp.get('what_to_ask_doctor', 'N/A')}")

---

## Domain 2: Medication Interpretation

For each medication, CareMap provides:
- What the medication does (plain language)
- Important timing notes
- Safety warnings from interaction notes
- What to watch for

In [ ]:
from caremap.medication_interpretation import interpret_medication_v3_grounded, MED_V3_OUT_KEYS

print("Interpreting medications...")
print("=" * 60)

med_results = []

for med in tqdm(patient_data['medications'][:4], desc="Medications"):  # First 4 meds
    result, raw = interpret_medication_v3_grounded(
        client=client,
        medication_name=med['medication_name'],
        sig_text=med['sig_text'],
        clinician_notes=med.get('clinician_notes', ''),
        interaction_notes=med.get('interaction_notes', ''),
    )
    med_results.append({
        'name': med['medication_name'],
        'timing': med['timing'],
        'interpretation': result
    })

print("\nDone!")

In [ ]:
# Display medication interpretations
for med in med_results:
    print(f"\n{'='*60}")
    print(f"MEDICATION: {med['name']} ({med['timing']})")
    print(f"{'='*60}")
    interp = med['interpretation']
    if 'raw_response' not in interp:
        print(f"What it does: {interp.get('what_it_does', 'N/A')}")
        print(f"How to take: {interp.get('how_to_take', 'N/A')}")
        print(f"Watch out for: {interp.get('watch_out_for', 'N/A')}")
    else:
        print(f"[Parse error - raw response available]")

---

## Domain 3: Imaging Interpretation

Radiology reports are particularly dense with jargon. CareMap:
- Explains what the scan looked at
- Summarizes findings in 2-3 plain sentences
- **Never uses the word "cancer"** (that's for the oncologist)
- Provides a follow-up question

Let's interpret a CT chest scan for a heart failure patient:

In [ ]:
from caremap.imaging_interpretation import interpret_imaging_report, IMAGING_OUT_KEYS

# Load a CT scenario that matches our patient (CHF patient with shortness of breath)
ct_file = project_root / 'examples' / 'golden_imaging_ct.json'
with open(ct_file) as f:
    ct_data = json.load(f)

# Use the first scenario (chest CT for CHF patient)
ct_scenario = ct_data['ct_scenarios'][0]

print(f"Study: {ct_scenario['study_type']}")
print(f"Indication: {ct_scenario['clinical_indication']}")
print(f"\nRadiology Report (raw):")
print(f"{ct_scenario['radiology_report']['impression']}")

In [ ]:
# Interpret the imaging report
print("\nInterpreting imaging report...")

imaging_result = interpret_imaging_report(
    client=client,
    study_type=ct_scenario['study_type'],
    report_text=ct_scenario['radiology_report']['impression'],
    flag=ct_scenario['flag'],
)

print(f"\n{'='*60}")
print(f"IMAGING INTERPRETATION")
print(f"{'='*60}")
print(f"Study type: {imaging_result.get('study_type', 'N/A')}")
print(f"What was done: {imaging_result.get('what_was_done', 'N/A')}")
print(f"Key finding: {imaging_result.get('key_finding', 'N/A')}")
print(f"Ask the doctor: {imaging_result.get('what_to_ask_doctor', 'N/A')}")

---

## Domain 4: Care Gap Interpretation

Care gaps are overdue or pending health actions. CareMap:
- Converts clinical language to action items
- Preserves urgency (Today / This Week / Later)
- Provides clear next steps

In [ ]:
from caremap.caregap_interpretation import interpret_caregap, CARE_OUT_KEYS

print("Interpreting care gaps...")
print("=" * 60)

caregap_results = []

for gap in tqdm(patient_data['care_gaps'], desc="Care Gaps"):
    result = interpret_caregap(
        client=client,
        item_text=gap['item_text'],
        next_step=gap['next_step'],
        time_bucket=gap['time_bucket'],
    )
    caregap_results.append({
        'original': gap['item_text'],
        'urgency': gap['time_bucket'],
        'interpretation': result
    })

print("\nDone!")

In [ ]:
# Display care gap interpretations grouped by urgency
for urgency in ['Today', 'This Week', 'Later']:
    gaps = [g for g in caregap_results if g['urgency'] == urgency]
    if gaps:
        print(f"\n{'='*60}")
        print(f"{urgency.upper()}")
        print(f"{'='*60}")
        for gap in gaps:
            interp = gap['interpretation']
            print(f"[ ] {interp.get('action_item', gap['original'])}")
            print(f"    Next step: {interp.get('next_step', 'N/A')}")

---

## Generate Complete Fridge Sheet

Now we combine all interpretations into a single-page caregiver aid.

In [ ]:
def generate_fridge_sheet(patient_data: Dict, lab_results: List, med_results: List, 
                          caregap_results: List, imaging_result: Dict = None) -> str:
    """Generate a single-page fridge sheet in Markdown format."""
    
    patient = patient_data['patient']
    contacts = patient_data['contacts']
    
    lines = []
    
    # Header
    lines.append(f"# CareMap Fridge Sheet: {patient['nickname']}")
    lines.append(f"**Age:** {patient['age_range']} | **Conditions:** {', '.join(patient['conditions_display'])}")
    lines.append("")
    lines.append("---")
    lines.append("")
    
    # Today's Priorities
    lines.append("## Today's Priorities")
    today_gaps = [g for g in caregap_results if g['urgency'] == 'Today']
    for gap in today_gaps:
        action = gap['interpretation'].get('action_item', gap['original'])
        lines.append(f"- [ ] **{action}**")
    if not today_gaps:
        lines.append("- No urgent items today")
    lines.append("")
    
    # This Week
    lines.append("## This Week")
    week_gaps = [g for g in caregap_results if g['urgency'] == 'This Week']
    for gap in week_gaps:
        action = gap['interpretation'].get('action_item', gap['original'])
        next_step = gap['interpretation'].get('next_step', '')
        lines.append(f"- [ ] {action}")
        if next_step:
            lines.append(f"  - *{next_step}*")
    lines.append("")
    
    # Medications
    lines.append("## Medications")
    for med in med_results:
        interp = med['interpretation']
        name = med['name']
        timing = med['timing']
        what_it_does = interp.get('what_it_does', '') if 'raw_response' not in interp else ''
        
        lines.append(f"**{name}** ({timing})")
        if what_it_does:
            # Truncate to keep it brief
            short_desc = what_it_does[:150] + ('...' if len(what_it_does) > 150 else '')
            lines.append(f"- {short_desc}")
        lines.append("")
    
    # Labs
    lines.append("## Recent Labs")
    for lab in lab_results:
        interp = lab['interpretation']
        name = lab['test_name']
        flag = lab['flag']
        what_it_means = interp.get('what_it_means', '')
        
        # Extract just the first sentence for brevity
        brief = what_it_means.split('.')[0] + '.' if what_it_means else f"Result: {flag}"
        lines.append(f"- **{name}**: {brief}")
    lines.append("")
    
    # Imaging (if available)
    if imaging_result:
        lines.append("## Recent Imaging")
        study = imaging_result.get('study_type', 'Imaging study')
        finding = imaging_result.get('key_finding', '')
        lines.append(f"**{study}**")
        if finding:
            lines.append(f"- {finding}")
        lines.append("")
    
    # Contacts
    lines.append("## Contacts")
    lines.append(f"- **Clinic:** {contacts['clinic_name']} - {contacts['clinic_phone']}")
    lines.append(f"- **Pharmacy:** {contacts['pharmacy_name']} - {contacts['pharmacy_phone']}")
    lines.append("")
    
    # Footer
    lines.append("---")
    lines.append("")
    lines.append("*This sheet is for information only. It does not provide medical advice.*")
    lines.append("*Always confirm decisions with your healthcare provider.*")
    
    return "\n".join(lines)

In [ ]:
# Generate the fridge sheet
fridge_sheet = generate_fridge_sheet(
    patient_data=patient_data,
    lab_results=lab_results,
    med_results=med_results,
    caregap_results=caregap_results,
    imaging_result=imaging_result,
)

# Display as formatted Markdown
display(Markdown(fridge_sheet))

In [ ]:
# Save the fridge sheet
output_path = project_root / 'outputs' / 'fridge_sheet_dadu.md'
output_path.parent.mkdir(exist_ok=True)

with open(output_path, 'w') as f:
    f.write(fridge_sheet)

print(f"Saved to: {output_path}")

---

## Validation & Quality

CareMap includes an integration test suite that validates outputs against golden specifications.

### Test Results Summary

| Domain | Tests | Passed | Pass Rate |
|--------|-------|--------|----------|
| Care Gaps | 30 | 28 | 93% |
| Imaging (CT) | 10 | 10 | 100% |
| Imaging (MRI) | 5 | 4 | 80% |
| Labs | 16 | 11 | 69% |
| Medications | 21 | 21 | 100% |
| **Total** | **82** | **74** | **90.2%** |

### Known Limitations

1. **Abbreviation Echoing**: MedGemma sometimes echoes medical abbreviations from input (e.g., "INR", "A1c") despite prompt instructions to use plain language.

2. **Cancer Term Leakage**: In rare cases with serious findings, the model may use the word "cancer" despite explicit prohibition.

3. **LLM Variability**: As with all LLM outputs, results may vary between runs. The 90% pass rate represents typical performance.

### Mitigation Strategies

- Post-processing validation can flag outputs with forbidden terms
- Human review recommended for critical cases
- Prompts include extensive forbidden term lists with plain-language alternatives

---

## Architecture Overview

```
Patient Data (JSON)          Prompt Templates          MedGemma 4B
       │                            │                       │
       ▼                            ▼                       ▼
┌─────────────┐              ┌─────────────┐         ┌─────────────┐
│ medications │──────────────│ med_prompt  │────────▶│  Generate   │
│ labs        │──────────────│ lab_prompt  │────────▶│  with       │
│ imaging     │──────────────│ img_prompt  │────────▶│  safety     │
│ care_gaps   │──────────────│ gap_prompt  │────────▶│  constraints│
└─────────────┘              └─────────────┘         └─────────────┘
                                                            │
                                                            ▼
                                                     ┌─────────────┐
                                                     │  Validate   │
                                                     │  JSON +     │
                                                     │  Constraints│
                                                     └─────────────┘
                                                            │
                                                            ▼
                                                     ┌─────────────┐
                                                     │ Fridge Sheet│
                                                     │ (Markdown)  │
                                                     └─────────────┘
```

### Key Design Decisions

1. **Domain-specific prompts**: Each interpretation type has specialized prompts with forbidden term lists

2. **JSON validation**: Outputs are validated for structure and content constraints

3. **Sentence limits**: Enforced to keep output scannable (e.g., 3 sentences max for imaging findings)

4. **Question format**: All "ask the doctor" fields must end with a question mark

---

## Next Steps

### Try the Interactive Demo

Visit our HuggingFace Space to try CareMap with your own patient data:

**[CareMap Demo](https://huggingface.co/spaces/YOUR_USERNAME/caremap)** *(coming soon)*

### Future Enhancements

1. **Multilingual Support**: Generate fridge sheets in Spanish, Chinese, and other languages
2. **PDF Export**: One-click printable PDF generation
3. **Voice Readout**: Audio version for visually impaired caregivers
4. **EHR Integration**: Direct connection to patient portals

---

## Acknowledgments

- **Google Health AI** for MedGemma
- **Kaggle** for the MedGemma Impact Challenge
- Family caregivers everywhere who inspired this work

---

*CareMap - Making healthcare information accessible, one fridge sheet at a time.*